In [ ]:
!pip install transformers[torch] datasets seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-p

# Training task Token classification: Tìm vị trí lỗi

## Token Classification

In [ ]:
import torch
import numpy as np
import pandas as pd
# torch.set_printoptions(threshold=float('inf'))
# np.set_printoptions(threshold=float('inf'))

In [ ]:
import transformers

print(transformers.__version__)

4.41.2


### Tokenizer

In [ ]:
model_checkpoint = 'vinai/phobert-base-v2'

In [ ]:
from transformers import AutoTokenizer, BertForTokenClassification, TrainingArguments, Trainer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

## Preprocessing the data

In [ ]:
df = pd.read_csv('data50k.csv')
df.tail(6)

,sentence,label
49994,"Giốngnhư kỳ thi THCS , một số thí sinh Đã bị t...",2000000001000001000000
49995,Có 625 công ty Rromg toàn khu vực hội đủ vtiêu...,00001000001000010
49996,"Tiến thoáiW lưỡng nan Ông huỳnh Văn Tấn , 61 t...",01000100000000000000010000000010
49997,Và đó chắc chắn sẽ lậ bước đệm để họ tự tin bư...,00000100000000100
49998,Tôi có thói quen tự bươn chải và khJngquan niệ...,00000000200000000000
49999,Đôi khi lẽ ra ông ấy phả có bài phát bểu thì t...,00000010001000000000010


In [ ]:
from datasets import Dataset
ds = Dataset.from_pandas(df)
ds = ds.train_test_split(test_size=0.2)
ds

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 10000
    })
})

In [ ]:
def demofunc(input_ids, label):
  tokens = tokenizer.convert_ids_to_tokens(input_ids)
  i = 0
  token_label = []
  for token in tokens:
    if token in ['<s>', '<unk>', '<pad>', '</s>']:
      token_label.append(-100)
    elif token[-2:] == '@@':
      token_label.append(int(label[i]))
    else:
      token_label.append(int(label[i]))
      i += 1
  assert len(token_label) == len(tokens)
  return token_label


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['sentence'], truncation=True)
    labels = []
    for i, lb in enumerate(examples['label']):
      input_ids = tokenized_inputs['input_ids'][i]
      labels.append(demofunc(input_ids, lb))

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

## Batching

In [ ]:
tokenize_and_align_labels(ds['train'][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[0, 2754, 570, 4, 71, 351, 18, 372, 5132, 8228, 8, 18, 10, 3014, 3808, 7038, 43451, 18075, 675, 49, 7, 327, 5, 2], [0, 2120, 97, 8, 110, 1224, 306, 7, 2334, 8595, 1555, 19012, 8855, 53, 210, 29, 1529, 13572, 1086, 4283, 5, 2], [0, 59480, 7382, 115, 14945, 45112, 232, 8, 28209, 10625, 128, 100, 525, 69, 4770, 1579, 30555, 8627, 12, 58, 6, 470, 84, 24090, 7038, 4770, 2528, 18242, 1111, 4439, 5, 2], [0, 3305, 1977, 24919, 175, 10, 4623, 33, 97, 24, 2274, 28, 618, 175, 1176, 10, 9901, 525, 66, 137, 49751, 33652, 5, 2], [0, 432, 37, 13, 108, 3700, 52, 43, 16, 24153, 13342, 72, 15, 7007, 113, 166, 1624, 20999, 1395, 138, 5, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0,

In [ ]:
tokenized_ds = ds.map(tokenize_and_align_labels, batched=True, remove_columns=['sentence', 'label'])

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

## Fine tuning

In [ ]:
model_checkpoint = 'vinai/phobert-base-v2'
from transformers import AutoTokenizer, AutoModelForTokenClassification, BertForTokenClassification, TrainingArguments, Trainer

model = BertForTokenClassification.from_pretrained(model_checkpoint, num_labels = 3)

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.w

In [ ]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
from sklearn.metrics import recall_score, precision_score, f1_score

def align_predictions(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)
    batch_size, seq_len = preds.shape
    labels = label_ids
    preds_list, labels_list = [], []

    for batch_index in range(batch_size):
        batch_preds = preds[batch_index]
        batch_labels = labels[batch_index]
        valid_mask = batch_labels != -100  # loại bỏ padding tokens

        # Lọc theo valid_mask
        batch_preds = batch_preds[valid_mask]
        batch_labels = batch_labels[valid_mask]

        preds_list.extend(batch_preds)
        labels_list.extend(batch_labels)

    return preds_list, labels_list

# Trong compute_metrics
def compute_metrics(p):
    predictions, labels = p
    preds_list, labels_list = align_predictions(predictions, labels)
    precision = precision_score(labels_list, preds_list, average='macro')
    recall = recall_score(labels_list, preds_list, average='macro')
    f1 = f1_score(labels_list, preds_list, average='macro')
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [ ]:
args = TrainingArguments(
    f"tandat_model",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    logging_steps = 250,
    weight_decay=0.01,
    save_strategy="steps",  # Lưu lại model sau mỗi bước huấn luyện
    save_steps=1000  # Lưu lại sau mỗi 1000 bước
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.350000,0.320036,0.773751,0.660172,0.675447
2,0.307500,0.303392,0.829931,0.657879,0.666656
3,0.263200,0.218542,0.857199,0.838074,0.846997
4,0.177500,0.172195,0.924867,0.866431,0.891223
5,0.153500,0.156704,0.919624,0.907695,0.913378
6,0.136200,0.151937,0.925961,0.910479,0.917920
7,0.128400,0.150228,0.932085,0.911614,0.921391
8,0.118700,0.145922,0.928604,0.921255,0.924889
9,0.112200,0.146891,0.932384,0.921119,0.926625
10,0.109000,0.147469,0.935348,0.920194,0.927557


TrainOutput(global_step=6250, training_loss=0.18875930541992186, metrics={'train_runtime': 4456.6234, 'train_samples_per_second': 89.754, 'train_steps_per_second': 1.402, 'total_flos': 1.275449993931456e+16, 'train_loss': 0.18875930541992186, 'epoch': 10.0})

In [ ]:
trainer.save_model("pt10")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import shutil
import os

# Tạo tên file zip, ví dụ 'folder_name.zip'
zip_file = "/content/gdrive/MyDrive/checkpoint-6000.zip"
# Đường dẫn thư mục bạn muốn nén
folder_path = "/content/tandat_model/checkpoint-6000"

# Nén thư mục
shutil.make_archive(zip_file.replace('.zip', ''), 'zip', folder_path)


'/content/gdrive/MyDrive/checkpoint-6000.zip'

In [ ]:
from google.colab import files

# Tải tệp ZIP xuống
files.download(zip_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Load model sau khi đã training

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
root =  '/content/gdrive/MyDrive/'

In [ ]:
import zipfile

zip_path = root + 'checkpoint-5000.zip'
extract_path = 'checkpoint'

# Mở tệp zip và giải nén
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [ ]:
model_checkpoint = 'vinai/phobert-base-v2'

In [ ]:
from transformers import AutoTokenizer, BertForTokenClassification

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [ ]:
pretrain_checkpoint = 'checkpoint'
pretrain_model = BertForTokenClassification.from_pretrained(pretrain_checkpoint)
pretrain_model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05

## Xác định vị trí lỗi

In [ ]:
import torch
import numpy as np

# Input: 1 câu bị sai chính tả, show = True nếu muốn in ra từ bị lỗi
# Output: list index của các từ dự đoán là lỗi, mask của câu sau khi dự đoán (0 - Từ đúng, 1 - Từ đơn sai, 2 - Từ sai do xóa dấu cách)
def GetErrorIndex(sentence, show = False):
    inputs = tokenizer(sentence, return_tensors="pt")
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    with torch.no_grad():
        outputs = pretrain_model(**inputs)

    output = np.argmax(outputs.logits, axis=-1).reshape(-1)
    pred = []
    temp = []
    for i, token in enumerate(tokens):
        # if i > 0 and show:
        #     print(token, output[i].item())
        if token not in ['<s>', '<unk>', '<pad>', '</s>']:
            temp.append(output[i].item())
            if token[-2:] != '@@':
                pred.append(round(sum(temp)/len(temp) +0.1))
                temp.clear()
    pred = np.array(pred)

    # Loại trường hợp một số bị dự đoán là sai -----vì số liệu thì k biết đúng hay sai------------------
    words = sentence.split()
    for _, word in enumerate(words):
        if pred[_] and word.isdigit(): pred[_] = 0

    if show:
      print("Error words: ", end = '')
      for err, word in zip(pred, sentence.split()):
        if err: print(word, end=', ')
      print()
    return np.where(pred != 0)[0], pred


In [ ]:
sentence = 'Ra leejnh tập trận bằng vũ khí hạtnhân chến thuâtj , ông Putin dường mhư muốn thểiện nỗi giận dữ trước những bình luận gần đây từ phương Tây về chiến sự Ukraine.'
GetErrorIndex(sentence, True)

Error words: leejnh, hạtnhân, chến, thuâtj, mhư, thểiện, Ukraine., 


(array([ 1,  7,  8,  9, 14, 16, 32]),
 array([0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]))

## Masked LM

In [ ]:
model_checkpoint = 'vinai/phobert-base-v2'

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [ ]:
!pip install py_vncorenlp

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.8 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4306 sha256=4b36ed8a3ffa4a37f29d6d7a3f394744b9c1dd20290b4f42e4775c2e42120cd5
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [ ]:
import py_vncorenlp

# Automatically download VnCoreNLP components from the original repository
# and save them in some local machine folder
py_vncorenlp.download_model(save_dir='/content')

# Load the word and sentence segmentation component
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/content')

In [ ]:
# Input: 1 câu sai chính tả
# Output: Một list chứa các list top các từ dự đoán cho mỗi vị trí đc <mask>
def phobert_MaskedLM(sentence, top = 10, show = False):
  # word segment------------------------
  # segment_sent = rdrsegmenter.word_segment(sentence)[0].replace('< mask >', '<mask>')
  segment_sents = rdrsegmenter.word_segment(sentence)
  segment_sent = " ".join(segment_sents).replace('< mask >', '<mask>') # Phòng trường hợp câu bị tách thành nhiều câu riêng
  if show:
    print(segment_sent)

  # tokenize ------------------------------
  inputs = tokenizer(segment_sent, return_tensors="pt")
  with torch.no_grad():
      logits = model(**inputs).logits
  mask_token_indexs = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

  # get top predict ------------------------
  top_predict = []
  for i, mask_idx in enumerate(mask_token_indexs):
    cur_pred = []
    # print(f"Dự đoán <mask> thứ {i + 1} --------------")
    top_predicted_token_ids = torch.topk(logits[0, mask_idx], k=top).indices.reshape(-1)
    for _, token_id in enumerate(top_predicted_token_ids):
      # print(f'Top {_ + 1}: ', tokenizer.decode([token_id]))
      cur_pred.append(tokenizer.decode([token_id]))

    top_predict.append(cur_pred)
  return top_predict

sentence = 'Ông Nguyễn Khắc Chúc đang làm việc tại đại <mask> Quốc gia Hà Nội .'
phobert_MaskedLM(sentence, 20)


[['hoc',
  'Đại_học',
  'học',
  'Thư_viện',
  'học_tập',
  'Học_@@',
  'Chính_trị',
  'chủng_viện',
  'đại_học',
  'điện',
  'trường',
  'Âm_nhạc',
  'ĐH',
  'Đại_Học',
  'ich',
  'Sứ_quán',
  'lí_do',
  'học_t@@',
  'chính_trị',
  'viện']]

## Bartpho

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer, MBartForConditionalGeneration

#BARTpho-syllable
bartpho_tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-syllable")
bartpho_model = MBartForConditionalGeneration.from_pretrained("vinai/bartpho-syllable")

config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

dict.txt:   0%|          | 0.00/360k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

In [ ]:
# Input: 1 câu sai chính tả
# Output: Một list chứa các list top các từ dự đoán cho mỗi vị trí đc <mask>
def bartpho_MaskedLM(sentence, top = 10, show = False):
  # tokenize ------------------------------
  inputs = bartpho_tokenizer(sentence, return_tensors="pt")
  with torch.no_grad():
      logits = bartpho_model(**inputs).logits
  mask_token_indexs = (inputs.input_ids == bartpho_tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

  # get top predict ------------------------
  top_predict = []
  for i, mask_idx in enumerate(mask_token_indexs):
    cur_pred = []
    # print(f"Dự đoán <mask> thứ {i + 1} --------------")
    top_predicted_token_ids = torch.topk(logits[0, mask_idx], k=top).indices.reshape(-1)
    for _, token_id in enumerate(top_predicted_token_ids):
      # print(f'Top {_ + 1}: ', bartpho_tokenizer.decode([token_id]))
      cur_pred.append(bartpho_tokenizer.decode([token_id]))

    top_predict.append(cur_pred)
  return top_predict

sentence = 'Ra lệnh tập trận bằng vũ khí <mask> <mask> chiến thuật , ông Putin dường <mask> muốn thể hiện nỗi giận dữ'
bartpho_MaskedLM(sentence, 20)


[['hạt',
  'thông',
  'tối',
  'chính',
  'hu',
  'chiến',
  'hiện',
  'mới',
  'nguyên',
  'và',
  'hạng',
  'tấn',
  'công',
  'mạnh',
  'hoá',
  'phòng',
  'không',
  'sát',
  ',',
  'quân'],
 ['-',
  'Ele',
  'Hammer',
  ',',
  'Ar',
  'Kat',
  '<unk>',
  'Z',
  'Hel',
  'Ton',
  'Bla',
  'V',
  'To',
  'Né',
  'để',
  '',
  'No',
  'a',
  'và',
  'ar'],
 ['như',
  'đang',
  'không',
  'chỉ',
  'muốn',
  'nghĩ',
  'có',
  'đã',
  'chừng',
  't',
  'n',
  'chẳng',
  'd',
  'nhưng',
  'hồ',
  'nào',
  'cũng',
  'dư',
  'đây',
  'đó']]

## Sửa lỗi type 2
- Lỗi mất space

In [ ]:
vocab = tokenizer.get_vocab()

In [ ]:
def SpellingCorrector_type2(err_word):
    # Tokenize the word
    tokens = tokenizer.tokenize(err_word)
    # print(tokens)
    new_word = err_word[:-len(tokens[-1])] + ' ' + tokens[-1]
    # print(new_word)
    err_ids, masked = GetErrorIndex(new_word)
    if len(err_ids) == 0:
      return True, err_word[:-len(tokens[-1])] + '_' + tokens[-1]
    else:
      return False, err_word

SpellingCorrector_type2('trongđó')

(False, 'trongđó')

## So sánh độ tương đồng

In [ ]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.4 MB/s eta 0:00:00


In [ ]:
!pip install Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 15.2 MB/s eta 0:00:00


In [ ]:
import Levenshtein
from unidecode import unidecode

def distance(pred, err, del_tone = True):
  if del_tone:
      pred = unidecode(pred)
      err = unidecode(err)

  sc = 0
  for c in pred:
    if c in err: sc += 1
  return Levenshtein.distance(pred, err) - sc

distance('buồl', 'tiếc')

4

## Sửa lỗi chính tả

In [ ]:
def SpellingCorrector_1sent(sentence, MaskedLM = bartpho_MaskedLM, top = 15):
  # Xác định vị trí lỗi
  err_idx, pred = GetErrorIndex(sentence)
  print(err_idx, pred)
  words = sentence.split()

  # Sửa lỗi loại 1 ---------------------------------------------
  for idx in err_idx:
    if pred[idx] != 2:
      err_word = words[idx]
      words[idx] = '<mask>'
      masked_sentence = ' '.join(words)

      top_pred = MaskedLM(masked_sentence, top=top)
      print(masked_sentence)
      print(top_pred)
      print()
      dist = [] # Lưu khoảng cách giữa từ lỗi và các từ dự đoán.
      for pred_word in top_pred[0]:
        if '_' not in pred_word and '@@' not in pred_word: # Lỗi loại 1 chỉ là từ đơn
          dist.append(distance(pred_word, err_word))
        else:
          dist.append(1000)

      new_word_id = np.argmin(np.array(dist))
      new_word = top_pred[0][new_word_id]
      words[idx] = new_word
      print( 'err_type 1: ' + err_word + " --->>> " + new_word)

  # Sửa lỗi loại 2
  # 2.1 -- Sửa lỗi bị xóa space
  masked = [0] * len(words)
  for idx in err_idx:
    if pred[idx] == 2:
      err_word = words[idx]
      chk, new_word = SpellingCorrector_type2(err_word)
      if chk:
        words[idx] = new_word # Chưa kiểm tra xem còn lỗi khác ngoài xóa space không
        masked[idx] = 1
      print( 'err_type 2: ' + err_word + " --->>> " + new_word , chk)
  new_sentence2 = ' '.join(words)

  # 2.2 -- Sửa lỗi bị xóa space và các lỗi khác nữa
  for idx in err_idx:
    if pred[idx] == 2 and masked[idx] == 0:
      err_word = words[idx]
      words[idx] = '<mask> <mask>'
  masked_sentence = ' '.join(words).replace('_', ' ')
  top_pred = MaskedLM(masked_sentence, top=top)
  # print(top_pred)

  # mask tất cả  -------------------------------
  words = new_sentence2.split()
  cnt = 0
  for idx in err_idx:
     if pred[idx] == 2 and masked[idx] == 0:
        err_word = words[idx]
        words[idx] = ""

        dist = [] # Lưu khoảng cách giữa từ lỗi và các từ dự đoán.
        _pred = top_pred[cnt] + top_pred[cnt + 1]

        for pred_word in _pred:
          if len(pred_word.split('_')) == 2:
            dist.append(distance(pred_word.replace('_', ''), err_word))
          else:
            dist.append(1000)

        new_word_id = np.argmin(np.array(dist))
        new_word = _pred[new_word_id]

        if len(new_word.split('_')) == 2:
          words[idx] = new_word
          cnt += 2
          print( 'err_type 2: ' + err_word + " --->>> " + words[idx])
          continue

        dist = [] # Lưu khoảng cách giữa từ lỗi và các từ dự đoán.
        for pred_word in top_pred[cnt]:
          dist.append(distance(pred_word, err_word))
        new_word_id = np.argmin(np.array(dist))
        new_word = top_pred[cnt][new_word_id]
        words[idx] += new_word + '_'
        cnt += 1

        dist = [] # Lưu khoảng cách giữa từ lỗi và các từ dự đoán.
        for pred_word in top_pred[cnt]:
          dist.append(distance(pred_word, err_word))
        new_word_id = np.argmin(np.array(dist))
        new_word = top_pred[cnt][new_word_id]
        words[idx] += new_word
        print( 'err_type 2: ' + err_word + " --->>> " + words[idx])
        cnt += 1

  #   # mask lần lượt  -------------------------------

  return " ".join(words).replace('_', ' ')

In [ ]:
sentence = 'Dự án đầu tư cải tạo , mởrộngg vành đai 2 trên cao vàa dưới thấp đoạn từ ngã TƯ Sở đến Cầu Giấy có tổng đầu tư trên 21 . 000 tỷ đồng , trong đó đoạn dưới thấp hơn 17 . 000 tỷ đồng và đoạn trên cao gần 3 . 900 tỷ đồng .'
SpellingCorrector_1sent(sentence, phobert_MaskedLM)

[ 7 13 22 23] [0 0 0 0 0 0 0 2 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Dự án đầu tư cải tạo , mởrộngg vành đai 2 trên cao <mask> dưới thấp đoạn từ ngã TƯ Sở đến Cầu Giấy có tổng đầu tư trên 21 . 000 tỷ đồng , trong đó đoạn dưới thấp hơn 17 . 000 tỷ đồng và đoạn trên cao gần 3 . 900 tỷ đồng .
[['và', ',', '-', '–', 'đoạn', 'kết_hợp', '&', 'đến', '+', 'đi', 'lẫn', 'giai_đoạn', 'và@@', ';', '/']]

err_type 1: vàa --->>> và
Dự án đầu tư cải tạo , mởrộngg vành đai 2 trên cao và dưới thấp đoạn từ ngã TƯ Sở đến <mask> Giấy có tổng đầu tư trên 21 . 000 tỷ đồng , trong đó đoạn dưới thấp hơn 17 . 000 tỷ đồng và đoạn trên cao gần 3 . 900 tỷ đồng .
[['cầu', 'Cầu', 'Bãi_@@', 'Cầu_@@', 'đi_cầu', 'chợ', 'Hoa_@@', 'Bến', 'Dầu_@@', 'Chợ', 'Ao_@@', 'Hàng', 'Dốc_@@', 'Quán_@@', 'cầu_@@']]

err_type 1: Cầu --->>> Cầu
Dự án đầu tư cải tạo , mởrộngg vành đai 2 trên cao và dưới thấp đoạn từ ngã TƯ Sở đến Cầu <mask> có tổng đầu tư trên 21 . 000 tỷ đồ

'Dự án đầu tư cải tạo , mở rộng vành đai 2 trên cao và dưới thấp đoạn từ ngã TƯ Sở đến Cầu Giẽ có tổng đầu tư trên 21 . 000 tỷ đồng , trong đó đoạn dưới thấp hơn 17 . 000 tỷ đồng và đoạn trên cao gần 3 . 900 tỷ đồng .'

In [ ]:
import re
def SpellingCorrector(text, MaskedLM = bartpho_MaskedLM, top = 20, show = False):
  # Tiền xử lý text
  tokens = re.findall(r'\w+|[^\w\s]', text) # Tách dấu câu ra
  text = " ".join(tokens)
  return SpellingCorrector_1sent(text, MaskedLM, top)

sentence = 'Để hệ thống có thê hiểu tốt hơn từ ngữ về pháp luâth (domain adaption), chúng tôi tiếp tục huấn luyện hệ thống (continual pre-training) với tập vnLawData ở phần 3'
SpellingCorrector(sentence)

[12 14 27 28 30 34] [0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 2 2 0 2 0 0 0 1 0 0
 0]
Để hệ thống có thê hiểu tốt hơn từ ngữ về pháp <mask> ( domain adaption ) , chúng tôi tiếp tục huấn luyện hệ thống ( continual pre - training ) với tập vnLawData ở phần 3
[['lý', 'luật', 'lệnh', 'lí', 'y', 'quyền', 'quy', 'chế', 'điển', 'định', 'nhân', 'thuật', 'nguyên', 'điều', 'qui', 'trị', 'triển', 'tắc', 'đồ', 'hàm']]

err_type 1: luâth --->>> luật
Để hệ thống có thê hiểu tốt hơn từ ngữ về pháp luật ( <mask> adaption ) , chúng tôi tiếp tục huấn luyện hệ thống ( continual pre - training ) với tập vnLawData ở phần 3
[['<unk>', 'v', '', 'Law', 'luật', 'legal', 'a', 'pháp', 'Legal', 'legal', 'the', 'read', 'more', 'ad', 'law', 'Learn', 'see', 'ví', 'continua', 'learn']]

err_type 1: domain --->>> continua
Để hệ thống có thê hiểu tốt hơn từ ngữ về pháp luật ( continua adaption ) , chúng tôi tiếp tục huấn luyện hệ thống ( continual pre - training ) với tập <mask> ở phần 3
[['2', '1', 'bài',

'Để hệ thống có thê hiểu tốt hơn từ ngữ về pháp luật ( continua adaption ) , chúng tôi tiếp tục huấn luyện hệ thống ( continua ing p re - learning ing ) với tập huấn ở phần 3'

In [ ]:
sentence = ['Ra leejnh tập trận bằng vũkhí hạt nhân chien thuat , ông Putin đường mhư muốn thể hiện nỗi giận dữ trước những bình luận gần đây từ phương Tây về chiến sự Ukraine .',
  'HàNội sẽ mở rộng đường Láng từ khoảng 21 m cả haii Chiều hiennay lên 53 , 5 m , đồng thời xxaay dựng đường trên caoo từ Ngã Tư Sở đến Cầu Giấy .',
  'Sở Giao thông Vận tải vừa có báo cáo UBND thành phố việc triển khai lập báo cáo đề xuất chủtrương đầu tư',
  'Dự án đầu tư cải tạo , mở rộng vanh dai 2 trên cao và dưới thấp đoạn từ ngã TƯ Sở đến Cầu Giấy có tổng đầu tư trên 21 . 000 tỷ đồng , trong đó đoạn dưới thấp hơn 17 . 000 tỷ đồng và đoạn trên cao gần 3 . 900 tỷ đồng .',
  'campuchia tuyên bố không tham vần thêm về kếnh Funan Techo',
  'TP HCM sẽ làm tuyếN đường " mẫu " về k gian đô thị ở khu trung tâm',
   'Loại PSG , Dortmund vào chung kết Champions League',
  '500 cổ vật quý hiếm , trongđó có 18 bảo vật quốcgia sẽ được trưng bày tại Bảo tàngg Hải Phòng để người dân tham quan từ ngày 11 / 5 . ',
  'Nỗi khoo chủ quán cà phee gặp khách ngồi " cắm rễ "',]

for sent in sentence:
  print(sent)
  print("câu đã sửa: ", SpellingCorrector(sent, bartpho_MaskedLM))
  print()

Ra leejnh tập trận bằng vũkhí hạt nhân chien thuat , ông Putin đường mhư muốn thể hiện nỗi giận dữ trước những bình luận gần đây từ phương Tây về chiến sự Ukraine .
[ 1  5  8  9 14] [0 1 0 0 0 2 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
err_type 1: leejnh --->>> lệnh
err_type 1: chien --->>> chiến
err_type 1: thuat --->>> thuật
err_type 1: mhư --->>> như
err_type 2: vũkhí --->>> vũ_khí True
câu đã sửa:  Ra lệnh tập trận bằng vũ khí hạt nhân chiến thuật , ông Putin đường như muốn thể hiện nỗi giận dữ trước những bình luận gần đây từ phương Tây về chiến sự Ukraine .

HàNội sẽ mở rộng đường Láng từ khoảng 21 m cả haii Chiều hiennay lên 53 , 5 m , đồng thời xxaay dựng đường trên caoo từ Ngã Tư Sở đến Cầu Giấy .
[ 0  9 11 13 22 26] [2 0 0 0 0 0 0 0 0 1 0 1 0 2 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0]
err_type 1: m --->>> m
err_type 1: haii --->>> hai
err_type 1: xxaay --->>> xây
err_type 1: caoo --->>> cao
err_type 2: HàNội --->>> Hà_Nội True
err_type 2: hiennay --->>> hi

In [ ]:
sent0 = '500 cổ vật quý hiếm , trongdo có 18 bảo vật quốcgia sẽ được trưng bày tại Bảo tàngg Hải Phòng để người dân tham quan từ ngày 11 / 5 .'
Predict0(sent0, top = 30, show = True)

Error words: trongdo, quốcgia, tàngg, 
500 cổ_vật quý_hiếm , <mask> có 18 bảo_vật <mask> sẽ được trưng_bày tại Bảo <mask> Hải_Phòng để người_dân tham_quan từ ngày 11 / 5 .
['đặc_biệt', 'trong', 'gồm', 'đó', 'tiêu_biểu', 'nổi_bật', 'cá_biệt', 'đồng_thời', 'bao_gồm', 'nhất_là', 'hiện', 'riêng', 'trước_mắt', 'thậm_chí', 'chủ_yếu', 'vinh_dự', 'và', 'ước_tính', 'dự_kiến', 'ngoài_ra', 'bước_đầu', 'cụ_thể', 'đặc_sắc', 'ít_nhất', 'điển_hình', 'tự_hào', 'trưng_bày', 'còn', 'xác_định', 'nhưng']
['quốc_gia', 'Quốc_gia', 'thế_giới', 'đặc_biệt', 'vô_giá', 'tiêu_biểu', 'quốc_tế', 'QG', 'Việt_Nam', 'Quốc_Gia', 'Hoàng_cung', ',', 'quý_hiếm', 'nổi_tiếng', 'quý_giá', 'nhà_nước', 'châu_Á', 'quan_trọng', 'linh_thiêng', 'hoàng_cung', 'duy_nhất', 'gia_truyền', 'văn_hoá', 'Nhà_nước', 'hiếm', 'thiêng_liêng', 'độc_đáo', 'độc_nhất_vô_nhị', 'thiêng', 'cổ_truyền']
['tàng', 'tháp', 'thiên', 'đình', 'di', 'vật', 'tượng', 'văn_hoá', 'thất', 'thực_vật', 'Vương_Duy_@@', 'khố', 'Tàng', 'cảnh', 'hộ', 'tồn', 'thành', 'ni

'500 cổ vật quý hiếm , trong có 18 bảo vật quốc gia sẽ được trưng bày tại Bảo tàng Hải Phòng để người dân tham quan từ ngày 11 / 5 .'

# Test trên Dataset

In [ ]:
import pandas as pd

df50k = pd.read_csv('/content/data50k.csv')

In [ ]:
df50k.head()

In [ ]:
for _, sentence in enumerate(df50k.sentence):
  print("Câu gốc: ", sentence)
  print("--->", SpellingCorrector(sentence, top=20))
  print()
  if _ > 10: break